# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [14]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices

In [16]:
df = pd.read_csv('previsao_de_renda.csv')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [34]:
df.columns = [col.lower().replace(' ', '_') for col in df.columns]

df['posse_de_veiculo'] = df['posse_de_veiculo'].astype(str)
df['posse_de_imovel'] = df['posse_de_imovel'].astype(str)

df['sexo'] = df['sexo'].astype('category')
df['posse_de_veiculo'] = df['posse_de_veiculo'].astype('category')
df['posse_de_imovel'] = df['posse_de_imovel'].astype('category')
df['tipo_renda'] = df['tipo_renda'].astype('category')
df['educacao'] = df['educacao'].astype('category')
df['estado_civil'] = df['estado_civil'].astype('category')
df['tipo_residencia'] = df['tipo_residencia'].astype('category')


formula_inicial = '''
    np.log(renda) ~ C(sexo, Treatment(reference='F')) + C(posse_de_veiculo, Treatment(reference='False')) + 
    C(posse_de_imovel, Treatment(reference='False')) + qtd_filhos + C(tipo_renda, Treatment(reference='Assalariado')) + 
    C(educacao, Treatment(reference='Superior completo')) + C(estado_civil, Treatment(reference='Casado')) + 
    C(tipo_residencia, Treatment(reference='Casa')) + idade + tempo_emprego + qt_pessoas_residencia
'''

y, X = dmatrices(formula_inicial, data=df, return_type='dataframe')

modelo_inicial = sm.OLS(y, X).fit()
modelo_inicial_summary = modelo_inicial.summary()
modelo_inicial_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Mon, 01 Jul 2024   Prob (F-statistic):               0.00
Time:                        19:10:39   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
================================================================================================================================================
                                                                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                        6.6344      0.218     30.373      0.000       6.206       7.063
C(sexo, Treatment(reference='F'))[T.M]                                           0.7874      0.015     53.723      0.000       0.759       0.816
C(posse_de_veiculo, Treatment(reference='False'))[T.True]                        0.0441      0.014      3.119      0.002       0.016       0.072
C(posse_de_imovel, Treatment(reference='False'))[T.True]                         0.0829      0.014      5.926      0.000       0.055       0.110
C(tipo_renda, Treatment(reference='Assalariado'))[T.Bolsista]                    0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda, Treatment(reference='Assalariado'))[T.Empresário]                  0.1551      0.015     10.387      0.000       0.126       0.184
C(tipo_renda, Treatment(reference='Assalariado'))[T.Pensionista]                -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda, Treatment(reference='Assalariado'))[T.Servidor público]            0.0576      0.022      2.591      0.010       0.014       0.101
C(educacao, Treatment(reference='Superior completo'))[T.Primário]               -0.0938      0.072     -1.298      0.194      -0.235       0.048
C(educacao, Treatment(reference='Superior completo'))[T.Pós graduação]           0.0133      0.142      0.093      0.926      -0.265       0.292
C(educacao, Treatment(reference='Superior completo'))[T.Secundário]             -0.1079      0.014     -7.763      0.000      -0.135      -0.081
C(educacao, Treatment(reference='Superior completo'))[T.Superior incompleto]    -0.1370      0.033     -4.195      0.000      -0.201      -0.073
C(estado_civil, Treatment(reference='Casado'))[T.Separado]                       0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil, Treatment(reference='Casado'))[T.Solteiro]                       0.2672      0.109      2.453      0.014       0.054       0.481
C(estado_civil, Treatment(reference='Casado'))[T.União]                         -0.0343      0.025     -1.361      0.174      -0.084       0.015
C(estado_civil, Treatment(reference='Casado'))[T.Viúvo]                          0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia, Treatment(reference='Casa'))[T.Aluguel]                       0.0442      0.054      0.815      0.415      -0.062       0.150
C(tipo_residencia, Treatment(reference='Casa'))[T.Com os pais]                   0.0184      0.029      0.625      0.532      -0.039       0.076
C(tipo_residencia, Treatment(reference

In [36]:
p_values = modelo_inicial.pvalues
maior_pvalor = p_values.idxmax()

partes_formula = formula_inicial.split('+')
partes_formula = [parte.strip() for parte in partes_formula if maior_pvalor.split('[')[0].strip() not in parte]
nova_formula = ' + '.join(partes_formula).replace('~ +', '~').strip()

y, X = dmatrices(nova_formula, data=df, return_type='dataframe')


modelo_atualizado = sm.OLS(y, X).fit()
modelo_atualizado_summary = modelo_atualizado.summary()
modelo_atualizado_summary


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     340.0
Date:                Mon, 01 Jul 2024   Prob (F-statistic):               0.00
Time:                        19:10:56   Log-Likelihood:                -13601.
No. Observations:               12427   AIC:                         2.724e+04
Df Residuals:                   12406   BIC:                         2.740e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================================
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                 6.5899      0.219     30.108      0.000       6.161       7.019
C(sexo, Treatment(reference='F'))[T.M]                                    0.7800      0.015     53.208      0.000       0.751       0.809
C(posse_de_veiculo, Treatment(reference='False'))[T.True]                 0.0533      0.014      3.769      0.000       0.026       0.081
C(posse_de_imovel, Treatment(reference='False'))[T.True]                  0.0856      0.014      6.110      0.000       0.058       0.113
C(tipo_renda, Treatment(reference='Assalariado'))[T.Bolsista]             0.2988      0.242      1.237      0.216      -0.175       0.772
C(tipo_renda, Treatment(reference='Assalariado'))[T.Empresário]           0.1651      0.015     11.079      0.000       0.136       0.194
C(tipo_renda, Treatment(reference='Assalariado'))[T.Pensionista]         -0.2513      0.242     -1.039      0.299      -0.725       0.223
C(tipo_renda, Treatment(reference='Assalariado'))[T.Servidor público]     0.0754      0.022      3.400      0.001       0.032       0.119
C(estado_civil, Treatment(reference='Casado'))[T.Separado]                0.3238      0.112      2.903      0.004       0.105       0.542
C(estado_civil, Treatment(reference='Casado'))[T.Solteiro]                0.2616      0.109      2.396      0.017       0.048       0.476
C(estado_civil, Treatment(reference='Casado'))[T.União]                  -0.0374      0.025     -1.481      0.139      -0.087       0.012
C(estado_civil, Treatment(reference='Casado'))[T.Viúvo]                   0.3601      0.116      3.107      0.002       0.133       0.587
C(tipo_residencia, Treatment(reference='Casa'))[T.Aluguel]                0.0369      0.054      0.680      0.496      -0.069       0.143
C(tipo_residencia, Treatment(reference='Casa'))[T.Com os pais]            0.0185      0.029      0.627      0.530      -0.039       0.076
C(tipo_residencia, Treatment(reference='Casa'))[T.Comunitário]           -0.0403      0.091     -0.443      0.658      -0.218       0.138
C(tipo_residencia, Treatment(reference='Casa'))[T.Estúdio]                0.1225      0.084      1.456      0.145      -0.042       0.287
C(tipo_residencia, Treatment(reference='Casa'))[T.Governamental]         -0.0209      0.039     -0.535      0.593      -0.098       0.056
qtd_filhos                                                               -0.2567      0.108     -2.372      0.018      -0.469      -0.045
idade                                                                     0.0049      0.001      6.185      0.000       0.003       0.006
tempo_emprego                                  

In [38]:
variaveis_removidas = []
max_iter = 10
iter_count = 0

while iter_count < max_iter:
    p_values = modelo_inicial.pvalues
    maior_pvalor = p_values.idxmax()
    
    if p_values[maior_pvalor] < 0.05:
        break
    
    
    maior_pvalor_main = maior_pvalor.split('[')[0]
    
    
    partes_formula = formula_inicial.split('+')
    partes_formula = [parte.strip() for parte in partes_formula if maior_pvalor_main not in parte]
    nova_formula = ' + '.join(partes_formula).replace('~ +', '~').strip()

    y, X = dmatrices(nova_formula, data=df, return_type='dataframe')   
    modelo_inicial = sm.OLS(y, X).fit() 
    print(f"Iteração {iter_count + 1}: variável removida = {maior_pvalor}")
    iter_count += 1


modelo_final_summary = modelo_inicial.summary()
modelo_final_summary


Iteração 1: variável removida = C(educacao, Treatment(reference='Superior completo'))[T.Pós graduação]
Iteração 2: variável removida = C(tipo_residencia, Treatment(reference='Casa'))[T.Comunitário]
Iteração 3: variável removida = C(educacao, Treatment(reference='Superior completo'))[T.Pós graduação]
Iteração 4: variável removida = C(tipo_residencia, Treatment(reference='Casa'))[T.Comunitário]
Iteração 5: variável removida = C(educacao, Treatment(reference='Superior completo'))[T.Pós graduação]
Iteração 6: variável removida = C(tipo_residencia, Treatment(reference='Casa'))[T.Comunitário]
Iteração 7: variável removida = C(educacao, Treatment(reference='Superior completo'))[T.Pós graduação]
Iteração 8: variável removida = C(tipo_residencia, Treatment(reference='Casa'))[T.Comunitário]
Iteração 9: variável removida = C(educacao, Treatment(reference='Superior completo'))[T.Pós graduação]
Iteração 10: variável removida = C(tipo_residencia, Treatment(reference='Casa'))[T.Comunitário]


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     363.0
Date:                Mon, 01 Jul 2024   Prob (F-statistic):               0.00
Time:                        19:11:10   Log-Likelihood:                -13569.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12407   BIC:                         2.733e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
================================================================================================================================================
                                                                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                        6.6391      0.218     30.414      0.000       6.211       7.067
C(sexo, Treatment(reference='F'))[T.M]                                           0.7893      0.015     53.999      0.000       0.761       0.818
C(posse_de_veiculo, Treatment(reference='False'))[T.True]                        0.0442      0.014      3.125      0.002       0.016       0.072
C(posse_de_imovel, Treatment(reference='False'))[T.True]                         0.0819      0.014      5.966      0.000       0.055       0.109
C(tipo_renda, Treatment(reference='Assalariado'))[T.Bolsista]                    0.2219      0.241      0.920      0.357      -0.251       0.695
C(tipo_renda, Treatment(reference='Assalariado'))[T.Empresário]                  0.1556      0.015     10.433      0.000       0.126       0.185
C(tipo_renda, Treatment(reference='Assalariado'))[T.Pensionista]                -0.3116      0.241     -1.292      0.196      -0.784       0.161
C(tipo_renda, Treatment(reference='Assalariado'))[T.Servidor público]            0.0589      0.022      2.653      0.008       0.015       0.102
C(educacao, Treatment(reference='Superior completo'))[T.Primário]               -0.0980      0.072     -1.359      0.174      -0.239       0.043
C(educacao, Treatment(reference='Superior completo'))[T.Pós graduação]           0.0124      0.142      0.087      0.931      -0.266       0.291
C(educacao, Treatment(reference='Superior completo'))[T.Secundário]             -0.1077      0.014     -7.761      0.000      -0.135      -0.080
C(educacao, Treatment(reference='Superior completo'))[T.Superior incompleto]    -0.1364      0.033     -4.181      0.000      -0.200      -0.072
C(estado_civil, Treatment(reference='Casado'))[T.Separado]                       0.3272      0.111      2.942      0.003       0.109       0.545
C(estado_civil, Treatment(reference='Casado'))[T.Solteiro]                       0.2683      0.109      2.464      0.014       0.055       0.482
C(estado_civil, Treatment(reference='Casado'))[T.União]                         -0.0329      0.025     -1.310      0.190      -0.082       0.016
C(estado_civil, Treatment(reference='Casado'))[T.Viúvo]                          0.3678      0.116      3.182      0.001       0.141       0.594
qtd_filhos                                                                      -0.2655      0.108     -2.460      0.014      -0.477      -0.054
idade                                                                            0.0052      0.001      6.579      0.000       0.004       0.007
tempo_emprego                         